# 🧪 SpectraMind V50 — 03 · Ablation & Tuning

**NeurIPS 2025 Ariel Data Challenge** · *CLI‑first, Hydra‑safe, reproducible*

This notebook drives **systematic ablations and hyperparameter tuning** over the SpectraMind V50 stack.
It favors the **`spectramind` CLI** (Typer) for all heavy‑lifting, and **reads artifacts** to visualize
and compare results. Every step is **deterministic where possible** and produces evidence (logs/JSON/HTML).

> Golden rule: the notebook mirrors what you'd do from the CLI and never hides transformations.


## 🎯 Objectives

- Run or resume **symbolic‑aware ablations** (smoothness, symbolic weights, entropy penalties, COREL/σ, etc.).
- Inspect and plot **GLL, RMSE/MAE, calibration, violation scores** from ablation artifacts.
- Generate **Markdown + HTML leaderboards**, and optionally a **Top‑N ZIP** bundle.
- Append run metadata to `v50_debug_log.md` for auditability.


## 🔧 Environment & Paths

In [ ]:
from pathlib import Path
import json, os, sys, platform
from datetime import datetime

ROOT = Path.cwd()
ART = ROOT / "artifacts"
ABLATE_DIR = ART / "ablation"
ABLATE_RUNS = ABLATE_DIR / "runs"
ABLATE_SUMMARY_JSON = ABLATE_DIR / "leaderboard.json"
ABLATE_SUMMARY_CSV  = ABLATE_DIR / "leaderboard.csv"
ABLATE_SUMMARY_MD   = ABLATE_DIR / "leaderboard.md"
ABLATE_SUMMARY_HTML = ABLATE_DIR / "leaderboard.html"
TOPN_ZIP            = ABLATE_DIR / "topN_bundle.zip"
LOG_MD              = ROOT / "v50_debug_log.md"

for d in [ART, ABLATE_DIR, ABLATE_RUNS]:
    d.mkdir(parents=True, exist_ok=True)

env = {
    "timestamp": datetime.now().isoformat(timespec="seconds"),
    "python": sys.version.replace("\n", " "),
    "platform": platform.platform(),
    "cwd": str(ROOT),
    "paths": {
        "ART": str(ART),
        "ABLATE_DIR": str(ABLATE_DIR),
        "ABLATE_RUNS": str(ABLATE_RUNS),
        "ABLATE_SUMMARY_JSON": str(ABLATE_SUMMARY_JSON),
        "ABLATE_SUMMARY_CSV": str(ABLATE_SUMMARY_CSV),
        "ABLATE_SUMMARY_MD": str(ABLATE_SUMMARY_MD),
        "ABLATE_SUMMARY_HTML": str(ABLATE_SUMMARY_HTML),
        "TOPN_ZIP": str(TOPN_ZIP),
        "LOG_MD": str(LOG_MD),
    }
}
print(json.dumps(env, indent=2))


## 🩺 Quick CLI sanity (optional)

In [ ]:
import shutil, subprocess

def check_cli(cmd="spectramind", args=["--version"]):
    exe = shutil.which(cmd)
    if not exe:
        print("⚠️ 'spectramind' CLI not found on PATH. Skipping CLI sanity check.")
        return {"available": False}
    try:
        out = subprocess.check_output([cmd] + args, stderr=subprocess.STDOUT, text=True, timeout=30)
        print(out)
        return {"available": True, "output": out}
    except Exception as e:
        print(f"⚠️ CLI call failed: {e}")
        return {"available": True, "error": str(e)}

cli_info = check_cli()
cli_info


## 🧰 Define Ablation Grids

Below are **example grids** focused on symbolic and physics knobs. Tune to match your configs.

- `loss.symbolic.smoothness.lambda`: smoothness penalty weight
- `loss.symbolic.asymmetry.lambda`: asymmetry penalty weight
- `loss.symbolic.nonneg.lambda`: non‑negativity hinge
- `training.scheduler.max_lr`: learning rate peak
- `training.augmentation.jitter`: FGS1 jitter augmentation on/off
- `uncertainty.corel.enable`: COREL σ calibration toggle
- `uncertainty.temperature.value`: temperature scaling
- `decoder.sigma.attn_fusion`: enable attention×symbolic fusion in σ head


In [ ]:
# You can edit these directly or build them programmatically.
AB_GRID = {
    "one_at_a_time": {
        "loss.symbolic.smoothness.lambda": [0.0, 0.01, 0.05, 0.1],
        "loss.symbolic.asymmetry.lambda":  [0.0, 0.01, 0.05],
        "loss.symbolic.nonneg.lambda":     [0.0, 0.05],
        "training.scheduler.max_lr":       [1e-4, 3e-4, 1e-3],
        "training.augmentation.jitter":    [False, True],
        "uncertainty.corel.enable":        [False, True],
        "uncertainty.temperature.value":   [1.0, 1.25, 1.5],
        "decoder.sigma.attn_fusion":       [False, True],
    },
    "cartesian": {
        # Keep cartesian grids modest to respect runtime budgets.
        "loss.symbolic.smoothness.lambda": [0.0, 0.05],
        "training.scheduler.max_lr":       [1e-4, 3e-4],
        "uncertainty.corel.enable":        [False, True],
    }
}
print(json.dumps(AB_GRID, indent=2))


## 🚀 Launch ablations via `spectramind ablate` (or load existing results)

In [ ]:
import json, subprocess, shutil, os, time

def run_ablate(mode="one-at-a-time",
               top_n=10,
               retries=1,
               parallel=2,
               md_out=None,
               html_out=None,
               csv_out=None,
               topn_zip=None):
    exe = shutil.which("spectramind")
    if not exe:
        print("⚠️ CLI not found; skipping ablation launch. You can run it in a terminal and re-run next cells.")
        return False
    cmd = [
        "spectramind", "ablate",
        "--mode", mode,
        "--retries", str(retries),
        "--parallel", str(parallel),
        "--top_n", str(top_n),
    ]
    if md_out:
        cmd += ["--md", str(md_out)]
    if html_out:
        cmd += ["--open_html", "false", "--html", str(html_out)]
    if csv_out:
        cmd += ["--csv", str(csv_out)]
    if topn_zip:
        cmd += ["--zip", str(topn_zip)]
    # Note: advanced flags (symbols, metrics, diagnostics) are provided by the CLI impl.
    print("Running:", " ".join(cmd))
    try:
        subprocess.check_call(cmd, timeout=18000)  # 5 hours cap; adjust to your infra
        return True
    except Exception as e:
        print("Ablation run failed/aborted:", e)
        return False

# Example: try a quick one-at-a-time sweep that writes leaderboard artifacts
ok = run_ablate(
    mode="one-at-a-time",
    top_n=10,
    retries=1,
    parallel=2,
    md_out=ABLATE_SUMMARY_MD,
    html_out=ABLATE_SUMMARY_HTML,
    csv_out=ABLATE_SUMMARY_CSV,
    topn_zip=TOPN_ZIP
)

if ok:
    print("✅ Ablation CLI triggered. Artifacts should appear under:", ABLATE_DIR)
else:
    print("⚠️ Skipped or failed to launch ablation via CLI.")


## 📊 Parse leaderboard & visualize top runs

In [ ]:
import json, csv, math
from pathlib import Path

def load_leaderboard(json_path=ABLATE_SUMMARY_JSON, csv_path=ABLATE_SUMMARY_CSV):
    data = None
    if json_path.exists():
        try:
            with open(json_path, "r", encoding="utf-8") as f:
                data = json.load(f)
        except Exception as e:
            print("Failed to parse JSON leaderboard:", e)
    rows = []
    if csv_path.exists():
        try:
            import pandas as pd
            df = pd.read_csv(csv_path)
            print("CSV leaderboard loaded with shape:", df.shape)
            rows = df.to_dict(orient="records")
        except Exception as e:
            print("Failed to parse CSV leaderboard via pandas:", e)
            # Fallback to Python csv
            try:
                with open(csv_path, newline="", encoding="utf-8") as f:
                    reader = csv.DictReader(f)
                    rows = list(reader)
                print("CSV leaderboard loaded via csv module:", len(rows), "rows.")
            except Exception as e2:
                print("CSV fallback failed:", e2)
    return data, rows

lb_json, lb_rows = load_leaderboard()
if lb_json is None and not lb_rows:
    print("No leaderboard artifacts found yet. Once ablations complete, re-run this cell.")
else:
    # Heuristic: find a numeric "gll" or "score" column for ranking
    import numpy as np
    import matplotlib.pyplot as plt

    # Select rows and numeric metric
    metric_key = None
    if lb_rows:
        keys = lb_rows[0].keys()
        for k in ["gll", "GLL", "score", "val_gll", "val_score", "mean_gll"]:
            if k in keys:
                metric_key = k
                break
    # Plot top 15 by metric (lower GLL is better; adjust sign if needed)
    if lb_rows and metric_key:
        # Convert values to float where possible
        def to_float(x):
            try: return float(x)
            except: return math.inf
        sorted_rows = sorted(lb_rows, key=lambda r: to_float(r.get(metric_key, math.inf)))
        top = sorted_rows[:15]
        labels = [r.get("run_id", r.get("short_id", f"r{i}")) for i, r in enumerate(top)]
        vals = [to_float(r.get(metric_key, math.inf)) for r in top]
        plt.figure(figsize=(8, 4))
        plt.bar(range(len(vals)), vals)
        plt.xticks(range(len(vals)), labels, rotation=45, ha="right")
        plt.ylabel(metric_key)
        plt.title("Top runs by leaderboard metric")
        plt.tight_layout()
        plt.show()
    else:
        print("Leaderboard parsed, but could not identify a numeric metric column to plot.")


## 🧰 Optional: rebuild leaderboard & Top‑N ZIP via CLI

In [ ]:
# If you ran custom ablations manually, you can regenerate summary artifacts here.
import shutil, subprocess

def rebuild_leaderboard(md_out=ABLATE_SUMMARY_MD, html_out=ABLATE_SUMMARY_HTML,
                        csv_out=ABLATE_SUMMARY_CSV, topn_zip=TOPN_ZIP, top_n=10):
    exe = shutil.which("spectramind")
    if not exe:
        print("⚠️ CLI not found; cannot rebuild leaderboard via CLI.")
        return False
    cmd = [
        "spectramind", "ablate", "--rebuild-only",
        "--top_n", str(top_n),
        "--md", str(md_out),
        "--open_html", "false", "--html", str(html_out),
        "--csv", str(csv_out),
        "--zip", str(topn_zip),
    ]
    print("Running:", " ".join(cmd))
    try:
        subprocess.check_call(cmd, timeout=3600)
        return True
    except Exception as e:
        print("Rebuild failed:", e)
        return False

# Example (safe to skip):
# ok = rebuild_leaderboard()
# print("Rebuild:", ok)


## 🧾 Append run metadata to `v50_debug_log.md`

In [ ]:
from datetime import datetime
entry = f"""### Notebook: 03_ablation_and_tuning.ipynb
- timestamp: {datetime.now().isoformat(timespec="seconds")}
- cwd: {ROOT}
- python: {platform.python_version()}
- actions:
  - env_init
  - ablate_cli_attempted: true
  - leaderboard_json_exists: {ABLATE_SUMMARY_JSON.exists()}
  - leaderboard_csv_exists: {ABLATE_SUMMARY_CSV.exists()}
  - leaderboard_md_exists: {ABLATE_SUMMARY_MD.exists()}
  - leaderboard_html_exists: {ABLATE_SUMMARY_HTML.exists()}
  - topN_zip_exists: {TOPN_ZIP.exists()}
"""

try:
    with open(LOG_MD, "a", encoding="utf-8") as f:
        f.write(entry + "\n")
    print(f"Appended notebook log entry to {LOG_MD}")
except Exception as e:
    print(f"⚠️ Could not append to {LOG_MD}: {e}")
